In [3]:
%matplotlib inline
# -*- coding: utf-8 -*-

import color_hist_p3 as ch
import importlib
importlib.reload(ch)

from collections import Counter, defaultdict
import codecs

In [4]:
colwords = ['blue','red','green','yellow','white','black','grey','pink','purple','orange','brown']
Xtrain,Xtest = ch.load_region_data(data_path='../data/')
#(w2f_train,w2f_test,colf_train,Xtrain_hist3d,Xtest_hist3d) = ch.load_region_data(feat_path='../../Preproc/PreProcOut/Xcolorhist3d_v1.npz')

(12373, 515)
Xtrain (11415, 515)
Xtest (1000, 515)


In [5]:
with codecs.open('../data/image_regions.txt','r') as f:
    target_regions = f.readlines()

In [6]:
## Überprüfung, ob 'image_regions' auch genauso aufrufbar ist, wie ich es mir vorstelle.
img_region = [img.split('.')[0] for img in target_regions]
print(img_region)
img = [reg.split('_')[0] for reg in img_region]
region = [reg.split('_')[1] for reg in img_region]
len(region)

['7456_2', '2025_2', '4785_5', '39946_1', '26044_3', '27207_7', '12275_1', '16145_2', '5124_5', '27363_3', '9689_10', '37080_5', '4121_10', '16093_3', '6302_2', '3140_1', '9507_3', '22747_2', '30908_3', '22346_3', '11437_4', '39264_1', '16755_1', '1972_3', '15693_4', '30260_5', '10067_1', '1851_2', '16380_5', '1277_6']


30

In [12]:
set(Xtrain[:,0])

{0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0}

In [16]:
colormodel = ch.train_region_model(Xtrain,M_test=Xtest)

Building model...
1000/1000 [==============================] - 0s 60us/step
Test score: [2.0483697316646574, 0.6080000021457672]


In [17]:
## zukünftig dann nur doch das verwenden:
#reuse_model(M_test=Xtest, saved_model='model.json', saved_weights='model.h5')

NameError: name 'reuse_model' is not defined

In [18]:
colormodel.metrics_names

['loss', 'acc']

In [19]:
##predictions gibt mir das gleiche zurück, wie in color_errors.ipynb die jacknife-Funktion
Xtest_regionpred_3d = ch.predict_color(colormodel, Xtest)

Collection Test Splits (1000, 523)


In [20]:
Xtest_regionpred_3d.shape

(1000, 523)

In [21]:
set(list(Xtest_regionpred_3d[:,0]))

{9.167099301876647e-10,
 4.005714870913835e-09,
 1.4640771262008911e-08,
 6.450155609627473e-08,
 6.097339451116568e-07,
 1.3748880292041576e-06,
 1.9524495655787177e-06,
 4.9390523599868175e-06,
 6.187376129673794e-06,
 6.514324468298582e-06,
 1.3783339454676025e-05,
 1.4207226740836632e-05,
 1.5818495739949867e-05,
 1.6477220924571157e-05,
 2.266107185278088e-05,
 2.584287358331494e-05,
 3.295730130048469e-05,
 3.3454263757448643e-05,
 3.78724726033397e-05,
 3.9940634451340884e-05,
 4.322007589507848e-05,
 4.3897543946513906e-05,
 4.56265224784147e-05,
 6.019268403179012e-05,
 6.1015321989543736e-05,
 6.196719186846167e-05,
 7.870667468523607e-05,
 8.277715096483007e-05,
 9.963464253814891e-05,
 0.00013091218715999275,
 0.00013241074339020997,
 0.00015170025289990008,
 0.00016140626394189894,
 0.00016775104450061917,
 0.00017393998859915882,
 0.00019039009930565953,
 0.00019099860219284892,
 0.00020574552763719112,
 0.00020773937285412103,
 0.0002115976094501093,
 0.00021484140597749

In [22]:
Xtest_pred = Xtest_regionpred_3d[:,:len(colwords)]

In [23]:
Xtest_pred.shape

(1000, 11)

In [24]:
Xtest_pred[0,:]

array([1.11473389e-01, 7.04467654e-01, 1.31881312e-02, 2.29077414e-02,
       2.55058296e-02, 6.89049140e-02, 2.88694678e-03, 5.06646559e-02,
       1.51320066e-07, 1.07863009e-07, 4.46218024e-07])

In [25]:
img_in_test = []
for img_reg in img_region:
    for ix in range(len(Xtest)):
        color=Xtest[ix][0]
        fileid = str(int(Xtest[ix][1]))
        regionid = str(int(Xtest[ix][2]))
        img = img_reg.split('_')[0]
        region = img_reg.split('_')[1]
        
        if fileid == img and regionid == region:
            img_in_test.append((fileid+'_'+regionid, Xtest_pred[ix], Xtest_pred.argmax(axis=1)[ix], color))
print(len(img_in_test), img_in_test)

31 [('7456_2', array([1.25968412e-01, 1.56284403e-02, 5.64570203e-02, 7.82665730e-01,
       1.05554368e-02, 2.33490951e-03, 3.98972956e-03, 2.40036007e-03,
       1.56788662e-08, 1.24601005e-08, 2.10509885e-08]), 3, 1.0), ('2025_2', array([3.75858285e-02, 1.01349251e-02, 1.11353993e-02, 4.57951695e-01,
       3.65445018e-01, 9.58570093e-02, 1.89548768e-02, 2.91609438e-03,
       4.01885382e-06, 7.11847815e-06, 7.92385163e-06]), 3, 3.0), ('4785_5', array([9.16709930e-10, 2.41093323e-09, 1.44836156e-06, 9.99998331e-01,
       2.45209492e-07, 1.11155507e-09, 1.12643478e-10, 4.01202780e-13,
       6.42019573e-32, 8.12264010e-32, 2.36401341e-31]), 3, 3.0), ('39946_1', array([3.25807512e-01, 1.55096054e-01, 2.75046695e-02, 1.03230841e-01,
       1.58545330e-01, 1.21072866e-01, 1.86909195e-02, 8.91619027e-02,
       2.12634564e-04, 2.70464749e-04, 4.06871142e-04]), 0, 1.0), ('26044_3', array([1.05066225e-03, 7.89418336e-05, 9.95061696e-01, 3.54502513e-03,
       1.90680395e-04, 3.83882216e-0

In [26]:
for ix in range(len(Xtest)):
    color=Xtest[ix][0]
    fileid = str(int(Xtest[ix][1]))
    regionid = str(int(Xtest[ix][2]))
    if fileid == '2025':
        print(fileid+'_'+regionid, Xtest_pred[ix], Xtest_pred.argmax(axis=1)[ix], color)

2025_2 [3.75858285e-02 1.01349251e-02 1.11353993e-02 4.57951695e-01
 3.65445018e-01 9.58570093e-02 1.89548768e-02 2.91609438e-03
 4.01885382e-06 7.11847815e-06 7.92385163e-06] 3 3.0


In [28]:
for img in img_in_test:
    print(img[0])

7456_2
2025_2
4785_5
39946_1
26044_3
27207_7
12275_1
16145_2
5124_5
27363_3
9689_10
37080_5
4121_10
16093_3
6302_2
3140_1
9507_3
22747_2
30908_3
22346_3
22346_3
11437_4
39264_1
16755_1
1972_3
15693_4
30260_5
10067_1
1851_2
16380_5
1277_6


## Analyse der menschlichen Farbzuweisung und der vorhergesagten Zuweisung
['blue','red','green','yellow','white','black','grey','pink','purple','orange','brown']

In [163]:
pred_tagged = []
color_dict = defaultdict(list)
for ix in range(len(Xtest)):
    color=Xtest[ix][0]
    color_dict[int(color)].append(Xtest_pred.argmax(axis=1)[ix])

In [65]:
color_dict2 = defaultdict(list)

for key in color_dict:
    color_dict2[int(key)].append(Counter(color_dict[key]))

In [67]:
color_dict2

defaultdict(list,
            {0: [Counter({0: 116, 1: 8, 2: 8, 3: 1, 4: 14, 5: 8, 7: 1})],
             1: [Counter({0: 5, 1: 89, 2: 2, 3: 6, 4: 9, 5: 7, 7: 5})],
             2: [Counter({0: 8, 1: 6, 2: 116, 3: 8, 4: 8, 5: 6})],
             3: [Counter({0: 2, 1: 4, 2: 9, 3: 57, 4: 7, 5: 3})],
             4: [Counter({0: 26, 1: 11, 2: 19, 3: 11, 4: 173, 5: 9, 6: 3})],
             5: [Counter({0: 12, 1: 7, 2: 3, 3: 2, 4: 10, 5: 49})],
             6: [Counter({0: 7, 1: 1, 2: 5, 4: 16, 5: 3, 6: 1})],
             7: [Counter({0: 2, 1: 8, 2: 3, 3: 1, 4: 4, 5: 2, 6: 1, 7: 7})],
             8: [Counter({0: 2, 1: 3, 3: 1, 4: 3, 5: 2, 7: 1})],
             9: [Counter({0: 2, 1: 6, 2: 2, 3: 10, 4: 3, 5: 5})],
             10: [Counter({0: 4, 1: 11, 2: 13, 3: 6, 4: 10, 5: 6, 6: 1})]})

In [166]:
acc_color = defaultdict(list)
alle = 0
for key in color_dict2:
    count_list=[]
    total = 0
    acc_color2 = dict()
    for label in color_dict2[key][0]:
        count_list.append((label,color_dict2[key][0][label]))
        total = total+color_dict2[key][0][label]
    alle += total
    for label,item in count_list:
        prozent = (item/total)
        acc_color2[label]=prozent
    acc_color[key].append(acc_color2)

In [160]:
acc_color

defaultdict(list,
            {0: [{0: 0.7435897435897436,
               1: 0.05128205128205128,
               2: 0.05128205128205128,
               3: 0.00641025641025641,
               4: 0.08974358974358974,
               5: 0.05128205128205128,
               7: 0.00641025641025641}],
             1: [{0: 0.04065040650406504,
               1: 0.7235772357723578,
               2: 0.016260162601626018,
               3: 0.04878048780487805,
               4: 0.07317073170731707,
               5: 0.056910569105691054,
               7: 0.04065040650406504}],
             2: [{0: 0.05263157894736842,
               1: 0.039473684210526314,
               2: 0.7631578947368421,
               3: 0.05263157894736842,
               4: 0.05263157894736842,
               5: 0.039473684210526314}],
             3: [{0: 0.024390243902439025,
               1: 0.04878048780487805,
               2: 0.10975609756097561,
               3: 0.6951219512195121,
               4: 0.0853658

## 100-facher Durchlauf der Klassifikation

In [5]:
onehundret = []

img_region = [img.split('.')[0] for img in target_regions]

for i in range(100):
    colormodel = ch.train_region_model(Xtrain,M_test=Xtest)
    Xtest_regionpred_3d = ch.predict_color(colormodel, Xtest)
    Xtest_pred = Xtest_regionpred_3d[:,:len(colwords)]
    for img_reg in img_region:
        for ix in range(len(Xtest)):
            color=Xtest[ix][0]
            fileid = str(int(Xtest[ix][1]))
            regionid = str(int(Xtest[ix][2]))
            img = img_reg.split('_')[0]
            region = img_reg.split('_')[1]
        
            if fileid == img and regionid == region:
                onehundret.append((fileid+'_'+regionid, Xtest_pred[ix], Xtest_pred.argmax(axis=1)[ix], color))

Building model...
1000/1000 [==============================] - 0s 35us/step
Test score: [2.0112646152973177, 0.6149999991655349]
Collection Test Splits (1000, 523)
Building model...
1000/1000 [==============================] - 0s 36us/step
Test score: [2.0327300760746003, 0.6109999972581863]
Collection Test Splits (1000, 523)
Building model...
1000/1000 [==============================] - 0s 36us/step
Test score: [2.058738296508789, 0.6099999978542328]
Collection Test Splits (1000, 523)
Building model...
1000/1000 [==============================] - 0s 36us/step
Test score: [2.0076422543525694, 0.6069999984502792]
Collection Test Splits (1000, 523)
Building model...
1000/1000 [==============================] - 0s 56us/step
Test score: [2.01838991355896, 0.6060000034570694]
Collection Test Splits (1000, 523)
Building model...
1000/1000 [==============================] - 0s 44us/step
Test score: [2.0239054310321807, 0.6130000034570694]
Collection Test Splits (1000, 523)
Building model...
1

In [8]:
import numpy as np
#np.save('labeling_100iterations.txt', onehundret)
onehundret_dict = np.load('labeling_100iterations.txt.npy')

(image_id, probabilities, highest_probab, refexp_color)

**nächster Schritt:**
Berechnen der Varianzen pro Image. Können wir evtl. die Ergebnisse der Eyetrackingsstudie (pro Bild) darauf anwenden?

*https://docs.scipy.org/doc/numpy/reference/generated/numpy.var.html

np.load('labeling_100iterations.txt.npy')

In [36]:
img_region = [img.split('.')[0] for img in target_regions]


In [30]:
onehundret_dict = onehundret_dict[()]

In [35]:
onehundret_dict

{0: ('7456_2',
  array([9.49764699e-02, 2.95411497e-02, 5.83453774e-02, 7.83299148e-01,
         2.40304228e-02, 4.34155529e-03, 2.91927648e-03, 2.54638027e-03,
         3.11349204e-08, 1.35565983e-07, 1.61800969e-07]),
  3,
  1.0),
 1: ('2025_2',
  array([2.64577419e-02, 8.15119781e-03, 1.16669694e-02, 4.39655304e-01,
         3.90830696e-01, 1.00454450e-01, 2.01591030e-02, 2.56489986e-03,
         1.19842298e-05, 2.54893275e-05, 2.23229890e-05]),
  3,
  3.0),
 2: ('4785_5',
  array([7.88538013e-10, 2.20288662e-10, 1.80440429e-06, 9.99992132e-01,
         6.09714061e-06, 1.65183671e-08, 3.81901843e-09, 6.63309522e-13,
         4.38842445e-28, 3.59148649e-26, 4.99365192e-26]),
  3,
  3.0),
 3: ('39946_1',
  array([3.06102961e-01, 1.71165198e-01, 3.02548148e-02, 7.85885528e-02,
         2.06379071e-01, 1.22885443e-01, 2.86983699e-02, 5.55085875e-02,
         1.08350519e-04, 1.45518177e-04, 1.63044460e-04]),
  0,
  1.0),
 4: ('26044_3',
  array([2.70628277e-03, 3.90678528e-04, 9.94589925

In [127]:
#'7456_2'
probablist_7456=[onehundret_dict[key][1] for key in onehundret_dict if onehundret_dict[key][0] == img_region[0]]
probablist_2025=[onehundret_dict[key][1] for key in onehundret_dict if onehundret_dict[key][0] == img_region[1]]
probablist_4785=[onehundret_dict[key][1] for key in onehundret_dict if onehundret_dict[key][0] == img_region[2]]
probablist_39946=[onehundret_dict[key][1] for key in onehundret_dict if onehundret_dict[key][0] == img_region[3]]
probablist_26044=[onehundret_dict[key][1] for key in onehundret_dict if onehundret_dict[key][0] == img_region[4]]
probablist_27207=[onehundret_dict[key][1] for key in onehundret_dict if onehundret_dict[key][0] == img_region[5]]
probablist_12275=[onehundret_dict[key][1] for key in onehundret_dict if onehundret_dict[key][0] == img_region[6]]
probablist_16145=[onehundret_dict[key][1] for key in onehundret_dict if onehundret_dict[key][0] == img_region[7]]
probablist_5124=[onehundret_dict[key][1] for key in onehundret_dict if onehundret_dict[key][0] == img_region[8]]
probablist_27363=[onehundret_dict[key][1] for key in onehundret_dict if onehundret_dict[key][0] == img_region[9]]
probablist_9689=[onehundret_dict[key][1] for key in onehundret_dict if onehundret_dict[key][0] == img_region[10]]
probablist_37080=[onehundret_dict[key][1] for key in onehundret_dict if onehundret_dict[key][0] == img_region[11]]
probablist_4121=[onehundret_dict[key][1] for key in onehundret_dict if onehundret_dict[key][0] == img_region[12]]
probablist_16093=[onehundret_dict[key][1] for key in onehundret_dict if onehundret_dict[key][0] == img_region[13]]
probablist_6302=[onehundret_dict[key][1] for key in onehundret_dict if onehundret_dict[key][0] == img_region[14]]
probablist_3140=[onehundret_dict[key][1] for key in onehundret_dict if onehundret_dict[key][0] == img_region[15]]
probablist_9507=[onehundret_dict[key][1] for key in onehundret_dict if onehundret_dict[key][0] == img_region[16]]
probablist_22747=[onehundret_dict[key][1] for key in onehundret_dict if onehundret_dict[key][0] == img_region[17]]
probablist_30908=[onehundret_dict[key][1] for key in onehundret_dict if onehundret_dict[key][0] == img_region[18]]
probablist_22346=[onehundret_dict[key][1] for key in onehundret_dict if onehundret_dict[key][0] == img_region[19]]
probablist_11437=[onehundret_dict[key][1] for key in onehundret_dict if onehundret_dict[key][0] == img_region[20]]
probablist_39264=[onehundret_dict[key][1] for key in onehundret_dict if onehundret_dict[key][0] == img_region[21]]
probablist_16755=[onehundret_dict[key][1] for key in onehundret_dict if onehundret_dict[key][0] == img_region[22]]
probablist_1972=[onehundret_dict[key][1] for key in onehundret_dict if onehundret_dict[key][0] == img_region[23]]
probablist_15693=[onehundret_dict[key][1] for key in onehundret_dict if onehundret_dict[key][0] == img_region[24]]
probablist_30260=[onehundret_dict[key][1] for key in onehundret_dict if onehundret_dict[key][0] == img_region[25]]
probablist_10067=[onehundret_dict[key][1] for key in onehundret_dict if onehundret_dict[key][0] == img_region[26]]
probablist_1851=[onehundret_dict[key][1] for key in onehundret_dict if onehundret_dict[key][0] == img_region[27]]
probablist_16380=[onehundret_dict[key][1] for key in onehundret_dict if onehundret_dict[key][0] == img_region[28]]
probablist_1277=[onehundret_dict[key][1] for key in onehundret_dict if onehundret_dict[key][0] == img_region[29]]

In [129]:
probablist_2025

[array([2.64577419e-02, 8.15119781e-03, 1.16669694e-02, 4.39655304e-01,
        3.90830696e-01, 1.00454450e-01, 2.01591030e-02, 2.56489986e-03,
        1.19842298e-05, 2.54893275e-05, 2.23229890e-05]),
 array([3.55230607e-02, 6.21178513e-03, 8.99953488e-03, 5.61490476e-01,
        2.83727467e-01, 9.07258168e-02, 1.24953017e-02, 8.18863511e-04,
        3.47205741e-06, 2.88206797e-06, 1.40470149e-06]),
 array([3.66251282e-02, 1.23594133e-02, 1.24073615e-02, 4.57149684e-01,
        3.67439300e-01, 9.15424153e-02, 1.90507825e-02, 3.39911133e-03,
        5.69244230e-06, 1.00547313e-05, 1.09602124e-05]),
 array([4.65286262e-02, 1.14489747e-02, 1.64070558e-02, 5.22709668e-01,
        3.16766292e-01, 4.93206307e-02, 3.40914726e-02, 2.71151541e-03,
        2.28748877e-06, 4.92669551e-06, 8.59843203e-06]),
 array([3.26515622e-02, 1.80876497e-02, 8.42483621e-03, 5.22621810e-01,
        3.15432370e-01, 8.21930990e-02, 1.93764623e-02, 1.20800233e-03,
        1.15604871e-06, 1.29123703e-06, 1.676335

### 7456 - RefExp: 1

In [117]:
blue = [i[0] for i in probablist_7456]
red = [i[1] for i in probablist_7456]
green = [i[2] for i in probablist_7456]
yellow = [i[3] for i in probablist_7456]
white = [i[4] for i in probablist_7456]
black = [i[5] for i in probablist_7456]
grey = [i[6] for i in probablist_7456]
pink = [i[7] for i in probablist_7456]
purple = [i[8] for i in probablist_7456]
orange = [i[9] for i in probablist_7456]
brown = [i[10] for i in probablist_7456]

In [118]:
print(np.var(blue))
print(np.var(red))
print(np.var(green))
print(np.var(yellow))
print(np.var(white))
print(np.var(black))
print(np.var(grey))
print(np.var(pink))
print(np.var(purple))
print(np.var(orange))
print(np.var(brown))

0.0011238841968717423
0.0001372691203808551
0.0003906694451299857
0.002146004823179288
7.691081825210406e-05
6.8874003875292196e-06
4.8536111296843895e-06
6.132364836429344e-06
1.36047141025926e-11
1.1247929303782345e-11
2.5255318432574637e-12


In [ ]:
print(np.mean(blue))
print(np.mean(red))
print(np.mean(green))
print(np.mean(yellow))
print(np.mean(white))
print(np.mean(black))
print(np.mean(grey))
print(np.mean(pink))
print(np.mean(purple))
print(np.mean(orange))
print(np.mean(brown))

### 2025 - RefExp: 3

In [131]:
blue = [i[0] for i in probablist_2025]
red = [i[1] for i in probablist_2025]
green = [i[2] for i in probablist_2025]
yellow = [i[3] for i in probablist_2025]
white = [i[4] for i in probablist_2025]
black = [i[5] for i in probablist_2025]
grey = [i[6] for i in probablist_2025]
pink = [i[7] for i in probablist_2025]
purple = [i[8] for i in probablist_2025]
orange = [i[9] for i in probablist_2025]
brown = [i[10] for i in probablist_2025]

In [133]:
print(np.var(blue))
print(np.var(red))
print(np.var(green))
print(np.var(yellow))
print(np.var(white))
print(np.var(black))
print(np.var(grey))
print(np.var(pink))
print(np.var(purple))
print(np.var(orange))
print(np.var(brown))

0.0002115024537190352
3.1913720048861143e-05
2.2145135685195114e-05
0.004162891134146069
0.003112475419255181
0.0003825280972949884
5.294723274157822e-05
3.16850511799614e-06
8.789888311961887e-11
5.402048205063283e-11
8.143945150798494e-11


In [ ]:
print(np.mean(blue))
print(np.mean(red))
print(np.mean(green))
print(np.mean(yellow))
print(np.mean(white))
print(np.mean(black))
print(np.mean(grey))
print(np.mean(pink))
print(np.mean(purple))
print(np.mean(orange))
print(np.mean(brown))

### 4785 - RefExp: 3

In [134]:
blue = [i[0] for i in probablist_4785]
red = [i[1] for i in probablist_4785]
green = [i[2] for i in probablist_4785]
yellow = [i[3] for i in probablist_4785]
white = [i[4] for i in probablist_4785]
black = [i[5] for i in probablist_4785]
grey = [i[6] for i in probablist_4785]
pink = [i[7] for i in probablist_4785]
purple = [i[8] for i in probablist_4785]
orange = [i[9] for i in probablist_4785]
brown = [i[10] for i in probablist_4785]

In [136]:
print(np.var(blue))
print(np.var(red))
print(np.var(green))
print(np.var(yellow))
print(np.var(white))
print(np.var(black))
print(np.var(grey))
print(np.var(pink))
print(np.var(purple))
print(np.var(orange))
print(np.var(brown))

2.0135104100237808e-14
3.0459415026437408e-15
6.528876179114939e-12
1.202092789753806e-11
6.503258077609762e-12
3.122552332356674e-15
5.45380821763121e-18
2.748389499814418e-20
2.3749895091739e-47
4.888859135166785e-46
2.8830020289042717e-47


In [138]:
print(np.mean(blue))
print(np.mean(red))
print(np.mean(green))
print(np.mean(yellow))
print(np.mean(white))
print(np.mean(black))
print(np.mean(grey))
print(np.mean(pink))
print(np.mean(purple))
print(np.mean(orange))
print(np.mean(brown))

4.506012712282223e-08
1.5257289020472778e-08
1.3364350779987432e-06
0.9999970579147339
1.538300804830861e-06
1.6380918524926123e-08
1.4922349230806277e-09
2.0464667712206167e-11
8.187448299765267e-25
3.969385395795153e-24
8.358114758428668e-25


### 39946 - RefExp: 1

### 26044 - RefExp: 2

### 27207 - RefExp: 10

### 27207 - RefExp: 10

### 27207 - RefExp: 10

### 27207 - RefExp: 10

### 27207 - RefExp: 10